# The reduced decision making model by Wong-Wang
The equations and parameters are described in the body, appendix and supplementary materials of the following paper:

<i>Wong, Kong-Fatt and Xiao-Jing Wang.</i> <b>"A recurrent network mechanism  of time integration in perceptual decisions"</b>. Journal of Neuroscience 26, no. 4 (2006)

In [321]:
import numpy as np
import random

#Single-cell input-output relation [Appendix & Supplementary material D]
def input_output_relation(state,a=270,b=108,d=0.154):
    return (a*state - b)/(1-np.exp(-d*(a*state - b)))
#H(x) = (a * x - b) / (1 - exp(-d*(a * x - b)))

#Noisy sources from each ionic channel 
def noisy_channel(noises,tau_AMPA = 2, s_noise=0.05):
    #a two-dimensional array of noisy displacements for the current run
    eta =  np.random.rand(2) * np.sqrt(tau_AMPA * (s_noise ** 2))
    #Inoise1' = -Inoise1 + eta1 * sqrt(tau_AMPA * s_noise^2)
    #Inoise2' = -Inoise2 + eta2 * sqrt(tau_AMPA * s_noise^2)
    return -noises + eta #[ionic_noises]

#Summing contributions to the hidden state (x) of each population
#x1 = JN11 * S1 - JN12 * S2 + I0 + I1 + Inoise1
#x2 = JN22 * S2 - JN21 * S1 + I0 + I2 + Inoise2

def build_hidden_states(couplings,current_states,inputs,ionic_noises,external_sources = I0):
    #returns a 2x1 array, each element being the hidden state activity (firing rate) of one excitatory population
    return couplings @ current_states + inputs + ionic_noises + external_sources

#yet to port/solve

#these two will become the function below
#S1' = -S1/tau_S + (1 - S1) * g * H(x1)/1000
#S2' = -S2/tau_S + (1 - S2) * g * H(x2)/1000

def update_states(hidden_states,current_states,g=0.641,tau_S = 100):
    return -1*(current_states/tau_S) + (np.ones(2) - current_states) * g * input_output_relation(hidden_states)/1000

In [337]:
#Parameters
mu_0 = 20 #input strength; motion strength
c = 0 #input coherence

#Parameters to the input-output function
a = 270 #[(VnC)^{-1}]
b = 108 #[Hz]
d = 0.154 #[s]

#Kinetic parameters
g = 0.641 #dimensionless, called gamma in the paper
tau_S = 100 #[ms]
tau_AMPA = 2 #[ms]

#Synaptic coupling (population autapses {11,22} and between populations {21,12})
#JN11 = 0.2609
#JN22 = 0.2609
self_coupling = 0.2609
#JN21 = 0.0497
#JN12 = 0.0497
outer_coupling = 0.0497
couplings = [self_coupling,-outer_coupling]
#a 2x2 matrix whose main diagonal is comprised of self_coupling and whose secondary diagonal is -outer_coupling
couplings_matrix = [couplings,couplings[::-1]]

#Inputs from other sources (inhibitory and neutral populations: JA_ext; external: I0)
JA_ext = 5.2*np.exp(-4)
I0 = 0.3255

#Simulation noise
s_noise=0

#Visual motion stimulus to to the i-th {1,2} population 
I1 = JA_ext * mu_0 * (1 + c/100)
I2 = JA_ext * mu_0 * (1 - c/100)
inputs = np.array([I1,I2])

#Initial activity (firing rate) of each population
S1 = 1.1
S2 = 1.1
population_states = np.array([S1,S2])
current_noise = np.zeros(2)

#Simulation
sim_steps = 20_000
dt = 0.001
timestamp = 0

for i in range(sim_steps):
    current_noise = noisy_channel(current_noise)
    hidden_states = build_hidden_states(couplings=couplings_matrix,current_states=population_states,inputs=inputs,
                                    ionic_noises=current_noise,external_sources = I0)
    #print(hidden_states)
    population_states = update_states(hidden_states = hidden_states,current_states=population_states)
    print(population_states)

[-0.04709381 -0.04678567]
[0.33259025 0.33337572]
[0.21994559 0.22019531]
[0.24729814 0.24662478]
[0.2465098  0.24855785]
[0.24128489 0.24537578]
[0.25332981 0.24690993]
[0.2407521  0.24107106]
[0.25334631 0.25442503]
[0.23359838 0.23929691]
[0.26177905 0.25013562]
[0.22802338 0.24402713]
[0.26005899 0.24769559]
[0.23302838 0.24278452]
[0.26190786 0.2508923 ]
[0.22706487 0.23542636]
[0.26996494 0.2534665 ]
[0.21879612 0.24135882]
[0.27310398 0.25048053]
[0.21826729 0.23860871]
[0.27196367 0.25693999]
[0.22452906 0.23462374]
[0.26561636 0.2602737 ]
[0.22514287 0.23445803]
[0.26722375 0.25446633]
[0.22100821 0.24183713]
[0.27613333 0.24627612]
[0.21633161 0.24146807]
[0.2781143  0.25473418]
[0.21804162 0.23854985]
[0.26868649 0.25614273]
[0.22442144 0.23674126]
[0.26811554 0.25467752]
[0.22799846 0.23948102]
[0.25769993 0.24820699]
[0.23476696 0.24384119]
[0.25409065 0.24309939]
[0.23652188 0.24611974]
[0.25390784 0.24656431]
[0.23401538 0.24716434]
[0.25924886 0.2461252 ]
[0.23191112 0.

[0.24067993 0.24617844]
[0.25468929 0.24563845]
[0.23558473 0.24205013]
[0.25453304 0.24723677]
[0.23374121 0.2466722 ]
[0.25817566 0.24681249]
[0.2356807  0.24651783]
[0.25351516 0.24192824]
[0.24088581 0.24899406]
[0.24860237 0.2439095 ]
[0.24690047 0.24681435]
[0.24164641 0.24153716]
[0.24606164 0.251439  ]
[0.2451272  0.24221388]
[0.24678768 0.25034513]
[0.24198918 0.24072062]
[0.25280067 0.2533849 ]
[0.23386314 0.23769783]
[0.26328794 0.25004444]
[0.22883788 0.24074381]
[0.25968298 0.25429896]
[0.22866986 0.2344866 ]
[0.25963823 0.25554662]
[0.23045694 0.24046382]
[0.26108927 0.24568683]
[0.22850544 0.24692666]
[0.26792179 0.24195881]
[0.22129443 0.24953245]
[0.2727324  0.23734968]
[0.22010723 0.25695917]
[0.268658   0.23052079]
[0.22166657 0.26655941]
[0.27539283 0.22782287]
[0.21729941 0.26050164]
[0.27656735 0.22781963]
[0.21651967 0.26540838]
[0.27524767 0.2257128 ]
[0.21777175 0.2672442 ]
[0.27832868 0.22868837]
[0.20893643 0.25849198]
[0.28394873 0.23764048]
[0.21317607 0.25

[0.27660259 0.23198612]
[0.21899596 0.25512439]
[0.26861533 0.23836863]
[0.22710791 0.25648854]
[0.26339852 0.23332559]
[0.22446434 0.25774718]
[0.27115046 0.23714526]
[0.21676199 0.25517013]
[0.27292664 0.23240441]
[0.22280403 0.25905173]
[0.26999967 0.2280441 ]
[0.22356771 0.26415981]
[0.27169085 0.22525104]
[0.22252795 0.26734908]
[0.2733701  0.22665604]
[0.21703591 0.26708699]
[0.27871333 0.22689102]
[0.21446596 0.26164656]
[0.28050952 0.23308101]
[0.21133852 0.25436013]
[0.28426484 0.24000089]
[0.20748034 0.25391147]
[0.28376324 0.2344691 ]
[0.21120418 0.26182438]
[0.27925512 0.2310993 ]
[0.21467788 0.25545583]
[0.27875635 0.23743523]
[0.21659633 0.25621675]
[0.27672959 0.23772098]
[0.21772741 0.25201519]
[0.26940913 0.23544252]
[0.22067331 0.25847265]
[0.2723549  0.22968017]
[0.21519559 0.26137881]
[0.28100769 0.23185297]
[0.20696483 0.26311132]
[0.28808333 0.22630774]
[0.20873492 0.26499503]
[0.284747   0.22895474]
[0.20418724 0.25986326]
[0.28895317 0.23132984]
[0.2063065  0.25

[0.21920242 0.1947066 ]
[0.27349237 0.29772802]
[0.21985388 0.20108202]
[0.27489631 0.28763685]
[0.21800614 0.20634945]
[0.27418705 0.28531786]
[0.21792956 0.20717416]
[0.27343097 0.28866765]
[0.21699935 0.20230851]
[0.27719267 0.28794977]
[0.21911413 0.20937552]
[0.26936683 0.27993303]
[0.21959154 0.2132947 ]
[0.27336605 0.28355658]
[0.21791978 0.20515256]
[0.27657719 0.28519057]
[0.21938443 0.20582676]
[0.26982075 0.28387052]
[0.2208617  0.20995145]
[0.27272195 0.2872019 ]
[0.21452611 0.20323367]
[0.2834635  0.29453095]
[0.20594502 0.19881209]
[0.28390139 0.30004496]
[0.20707584 0.19135723]
[0.28225119 0.30426748]
[0.21118457 0.18855199]
[0.28313668 0.3082652 ]
[0.20643011 0.1836609 ]
[0.29213662 0.31540757]
[0.19755209 0.18265647]
[0.29540303 0.31180702]
[0.19793529 0.18443563]
[0.29308332 0.30963265]
[0.20356628 0.18608685]
[0.29067276 0.31341682]
[0.19983144 0.18302043]
[0.29318308 0.31647326]
[0.19857351 0.18373142]
[0.29201111 0.30964916]
[0.20712272 0.18860568]
[0.28839725 0.30

[0.18499947 0.22235644]
[0.31307858 0.26761504]
[0.18245538 0.22534503]
[0.31107546 0.26239444]
[0.18418415 0.23015756]
[0.31710235 0.26005813]
[0.17811271 0.22832275]
[0.32407013 0.26470686]
[0.17323368 0.22272979]
[0.33072054 0.26638498]
[0.16710972 0.22324259]
[0.32946491 0.2732137 ]
[0.17526163 0.22026301]
[0.31850503 0.27175958]
[0.17892102 0.22171276]
[0.32188066 0.26915034]
[0.17399041 0.22073071]
[0.32481563 0.27631119]
[0.17590508 0.21326834]
[0.3239564  0.27950186]
[0.17648609 0.21048066]
[0.32203535 0.27997439]
[0.17345494 0.2096372 ]
[0.33029944 0.28129383]
[0.170934   0.20935315]
[0.33060867 0.2881898 ]
[0.16919541 0.19983338]
[0.32785759 0.29520109]
[0.17274731 0.19695591]
[0.3267538 0.2994403]
[0.17456527 0.19234062]
[0.32746893 0.3020335 ]
[0.17117144 0.18908124]
[0.32858255 0.30805888]
[0.16990293 0.18395167]
[0.33094073 0.31777623]
[0.16666883 0.18121587]
[0.33410495 0.31438206]
[0.16453141 0.18667594]
[0.33415307 0.30355373]
[0.16445663 0.19683059]
[0.33647024 0.2963

...

[-0.00740918  0.21166519]
[0.74451859 0.27346472]
[-0.00744518  0.2233806 ]
[0.74154679 0.26870431]
[-0.00741546  0.22359251]
[0.73884816 0.26357243]
[-0.00738848  0.22991854]
[0.74113204 0.2574283 ]
[-0.00741132  0.2328449 ]
[0.73573424 0.25270367]
[-0.00735734  0.24162017]
[0.73815642 0.24588435]
[-0.00738156  0.24062833]
[0.73371864 0.24702716]
[-0.00733718  0.24122344]
[0.74036908 0.24835406]
[-0.00740369  0.24417638]
[0.74671545 0.24630207]
[-0.00746715  0.2426585 ]
[0.74864182 0.24584776]
[-0.00748642  0.24570613]
[0.74869919 0.24253051]
[-0.00748699  0.24355497]
[0.74419077 0.24887373]
[-0.00744191  0.23882513]
[0.7521565  0.24894168]
[-0.00752156  0.24362556]
[0.74296429 0.24942005]
[-0.00742964  0.24388837]
[0.74093573 0.2410525 ]
[-0.00740936  0.2508494 ]
[0.748677   0.23507109]
[-0.00748677  0.25530504]
[0.75372421 0.23781481]
[-0.00753724  0.24801667]
[0.76259333 0.24544223]
[-0.00762593  0.24406962]
[0.76789681 0.24541998]
[-0.00767897  0.24078033]
[0.76334439 0.24784439]


[-0.0081494   0.23517519]
[0.81766519 0.25436349]
[-0.00817665  0.23541359]
[0.82273729 0.25128193]
[-0.00822737  0.23905626]
[0.81069529 0.25047088]
[-0.00810695  0.24192478]
[0.80943855 0.24745805]
[-0.00809439  0.24044011]
[0.80718305 0.2529498 ]
[-0.00807183  0.2365801 ]
[0.81053324 0.25271824]
[-0.00810533  0.23464202]
[0.81246534 0.25795465]
[-0.00812465  0.23551392]
[0.81215889 0.25486669]
[-0.00812159  0.23234232]
[0.81916584 0.26042131]
[-0.00819166  0.23204099]
[0.81701067 0.25408284]
[-0.00817011  0.23700434]
[0.82050858 0.25554498]
[-0.00820509  0.23209779]
[0.81789644 0.26000547]
[-0.00817896  0.2294921 ]
[0.8111138  0.26323333]
[-0.00811114  0.22574536]
[0.80373331 0.26591435]
[-0.00803733  0.22243758]
[0.79684869 0.26471275]
[-0.00796849  0.22731276]
[0.80183493 0.2594678 ]
[-0.00801835  0.23181142]
[0.7989565  0.25405688]
[-0.00798957  0.23713156]
[0.80421115 0.2518945 ]
[-0.00804211  0.24138397]
[0.80904042 0.24770546]
[-0.0080904   0.23956431]
[0.81793199 0.24913542]


[-0.00745981  0.16112908]
[0.7443329  0.33847668]
[-0.00744333  0.15926042]
[0.73361932 0.34283032]
[-0.00733618  0.15503864]
[0.73073496 0.35192197]
[-0.00730733  0.15173426]
[0.73418997 0.35244872]
[-0.00734187  0.154459  ]
[0.7310085  0.35016937]
[-0.00731008  0.15650341]
[0.73185015 0.34535765]
[-0.0073185   0.16013522]
[0.72984127 0.34096535]
[-0.0072984   0.15925642]
[0.73515173 0.34687246]
[-0.00735151  0.15221176]
[0.74222589 0.34978646]
[-0.00742226  0.15680504]
[0.73869313 0.34247868]
[-0.00738693  0.15780736]
[0.73731801 0.34248897]
[-0.00737317  0.16172272]
[0.73525313 0.3375296 ]
[-0.00735253  0.16502436]
[0.73842433 0.32983497]
[-0.00738423  0.16852449]
[0.7278998  0.32905007]
[-0.00727893  0.17135826]
[0.7255925  0.32267425]
[-0.00725587  0.17734362]
[0.72259276 0.3208289 ]
[-0.00722581  0.17676597]
[0.71761486 0.31931935]
[-0.00717595  0.17782666]
[0.71614208 0.32034991]
[-0.00716062  0.17269501]
[0.70820337 0.32410092]
[-0.00708042  0.17530718]
[0.71186304 0.32121547]


[0.60037918 0.13290601]
[0.01482563 0.37913447]
[0.59927203 0.13052457]
[0.01821946 0.38029083]
[0.59281868 0.13075227]
[0.01705429 0.38139265]
[0.60073542 0.12688032]
[0.01290214 0.38339187]
[0.60853387 0.12814228]
[0.01061507 0.39058327]
[0.6148326  0.12543622]
[0.00911077 0.3831972 ]
[0.61927378 0.13317471]
[0.00674965 0.37880606]
[0.62037158 0.12933067]
[0.01057194 0.38764401]
[0.60790227 0.12286043]
[0.01230058 0.39895477]
[0.60731887 0.11665163]
[0.01048427 0.40390133]
[0.61310774 0.11312826]
[0.01080609 0.41085405]
[0.61256083 0.11276171]
[0.01146273 0.41118881]
[0.61066583 0.1067372 ]
[0.00941274 0.41811797]
[0.61662039 0.10845191]
[0.00926508 0.41830046]
[0.61495751 0.10162847]
[0.00835398 0.42360164]
[0.62269119 0.10084626]
[0.00734247 0.42318682]
[0.6271586  0.10097857]
[0.00354281 0.42144459]
[0.63755123 0.10497832]
[0.00255488 0.42074417]
[0.63543524 0.10733754]
[0.00279318 0.41768934]
[0.63270226 0.10877449]
[0.00398593 0.40895357]
[0.62812446 0.1152542 ]
[0.00636425 0.39

[0.03758027 0.30056816]
[0.54604837 0.19280159]
[0.03488292 0.30568303]
[0.55297405 0.18662791]
[0.03554584 0.30957913]
[0.54565916 0.18972058]
[0.03891599 0.30605781]
[0.53823439 0.1852038 ]
[0.04134365 0.313337  ]
[0.54159855 0.18549955]
[0.03873621 0.31223667]
[0.54882861 0.1803202 ]
[0.03334051 0.32247137]
[0.56033663 0.17675024]
[0.03201977 0.32529793]
[0.55329406 0.16812002]
[0.03252854 0.33652077]
[0.56110251 0.1652397 ]
[0.0327869  0.33152972]
[0.55909293 0.16670389]
[0.02879588 0.33049436]
[0.56963057 0.17085796]
[0.02529743 0.32444002]
[0.57264623 0.17662069]
[0.0247638  0.32027254]
[0.57990853 0.17375021]
[0.02128732 0.32143215]
[0.5880348  0.17387814]
[0.01991679 0.32557791]
[0.58582368 0.17062488]
[0.02339299 0.33167617]
[0.57378595 0.16719816]
[0.02577203 0.33015533]
[0.57399276 0.17319429]
[0.02790116 0.32789288]
[0.56572417 0.17244942]
[0.02886706 0.32235044]
[0.56784577 0.17336278]
[0.02578478 0.33010968]
[0.57202303 0.16645853]
[0.02458711 0.33781243]
[0.57963734 0.16

In [333]:
couplings_matrix @ population_states + I0

array([0.4523505 , 0.30241937])

[0.05628173 0.01153177]
[0.00264443 0.00565406]
[0.01339499 0.00111538]
[0.03864053 0.0042955 ]
[-0.03803393  0.03086786]
[ 0.10509467 -0.00710451]
[-0.05960245  0.04020571]
[0.09747971 0.02987598]
[-0.05928125  0.02041075]
[0.12855009 0.0317954 ]
[-0.09620062 -0.02469917]
[0.16669025 0.08190795]
[-0.1222719  -0.02965963]
[0.14809579 0.06438292]
[-0.11043049 -0.05436794]
[0.17530976 0.08904129]
[-0.10985199 -0.05085133]
[0.13868502 0.08402068]
[-0.13298254 -0.07186935]
[0.1581305  0.11493354]
[-0.13507207 -0.10183711]
[0.20480928 0.12372968]
[-0.15639185 -0.10374864]
[0.22626832 0.11044512]
[-0.19302945 -0.05541393]
[0.26109322 0.11272719]
[-0.22835541 -0.06381709]
[0.28511257 0.0839222 ]
[-0.22017516 -0.05759255]
[0.25716174 0.11763743]
[-0.2100648  -0.09061205]
[0.27106963 0.10127686]
[-0.21638309 -0.04863097]
[0.27179576 0.06508814]
[-0.22755469 -0.03071863]
[0.26066314 0.04017778]
[-0.2007307  -0.02357641]
[0.21767013 0.07889495]
[-0.16394741 -0.06461249]
[0.23260783 0.11397076]
[-

[ 0.2887661  -0.31112642]
[-0.22900066  0.34352562]
[ 0.27791258 -0.29217183]
[-0.22235764  0.35835891]
[ 0.29288327 -0.295729  ]
[-0.23436551  0.30525276]
[ 0.28391537 -0.26532327]
[-0.22762664  0.31879994]
[ 0.28963194 -0.28562874]
[-0.27652788  0.33061116]
[ 0.32787447 -0.31863275]
[-0.30566503  0.32135686]
[ 0.30623638 -0.29326493]
[-0.26018485  0.32558126]
[ 0.32132068 -0.31033905]
[-0.3006994   0.31065569]
[ 0.36945021 -0.27799518]
[-0.33534678  0.34620455]
[ 0.37489328 -0.30606211]
[-0.34069945  0.30779073]
[ 0.3464207  -0.29182591]
[-0.30557104  0.35782388]
[ 0.34115673 -0.34559911]
[-0.3212219   0.37532597]
[ 0.35455718 -0.33425007]
[-0.28938591  0.40450071]
[ 0.35844201 -0.36596138]
[-0.3365167   0.39112024]
[ 0.37128718 -0.38644297]
[-0.35978901  0.4233304 ]
[ 0.39093724 -0.40840312]
[-0.35477833  0.43238507]
[ 0.35634134 -0.39089193]
[-0.35205497  0.42062122]
[ 0.40233901 -0.39861797]
[-0.38522064  0.43311041]
[ 0.41481005 -0.4253044 ]
[-0.35641846  0.45439317]
[ 0.42024139

[-0.41764162  0.66032776]
[ 0.45698743 -0.6287054 ]
[-0.44034792  0.66078917]
[ 0.4667083  -0.63798314]
[-0.43050858  0.6899781 ]
[ 0.47115448 -0.67209229]
[-0.42392706  0.70051343]
[ 0.48137842 -0.65651178]
[-0.45441009  0.69089001]
[ 0.48450831 -0.64691823]
[-0.48224084  0.64884783]
[ 0.51589851 -0.64682411]
[-0.458132    0.65305723]
[ 0.50993006 -0.62491706]
[-0.4589586   0.68283402]
[ 0.52292261 -0.6302144 ]
[-0.518574    0.66016581]
[ 0.57758451 -0.63070156]
[-0.54867866  0.69922721]
[ 0.60478536 -0.67066918]
[-0.53811745  0.69599359]
[ 0.58186626 -0.69394627]
[-0.51300408  0.73687055]
[ 0.5660652  -0.70867217]
[-0.55157584  0.74266362]
[ 0.61772278 -0.70755365]
[-0.58602517  0.75367243]
[ 0.61677769 -0.6994759 ]
[-0.60139738  0.72935308]
[ 0.60228719 -0.67769104]
[-0.59347057  0.69876415]
[ 0.59894582 -0.67345113]
[-0.56805715  0.72707688]
[ 0.57146441 -0.71400024]
[-0.52377693  0.76014823]
[ 0.54808692 -0.74276148]
[-0.5090311   0.74319525]
[ 0.57479714 -0.69114022]
[-0.50458809

[-0.6578564   0.90122127]
[ 0.70251847 -0.88607375]
[-0.68173452  0.93482265]
[ 0.74830179 -0.91734041]
[-0.69879584  0.94860766]
[ 0.70861311 -0.8789336 ]
[-0.68917965  0.88652093]
[ 0.7288852  -0.83098234]
[-0.66952356  0.87846154]
[ 0.70883628 -0.82794917]
[-0.6696282   0.88038547]
[ 0.68957354 -0.84716684]
[-0.62700644  0.88632368]
[ 0.62753605 -0.81668681]
[-0.57407419  0.8684766 ]
[ 0.63621239 -0.86714485]
[-0.60386223  0.8893322 ]
[ 0.63203547 -0.84202937]
[-0.57248166  0.85616473]
[ 0.59934023 -0.82281997]
[-0.56032036  0.86016863]
[ 0.6007981 -0.8385176]
[-0.58061792  0.84272207]
[ 0.64946973 -0.81135996]
[-0.64319934  0.86330368]
[ 0.67371232 -0.84416792]
[-0.6256493   0.87627686]
[ 0.66818041 -0.86736116]
[-0.61750694  0.89342935]
[ 0.68307468 -0.86940888]
[-0.63331079  0.88770761]
[ 0.69002749 -0.87452803]
[-0.68933481  0.94192929]
[ 0.70380908 -0.90797682]
[-0.64646355  0.96100285]
[ 0.64745901 -0.90778495]
[-0.60960805  0.93277985]
[ 0.63148438 -0.87436086]
[-0.60827855  

[ 0.21165407 -0.51868293]
[-0.16725918  0.52474242]
[ 0.23416694 -0.47828072]
[-0.18513702  0.48216063]
[ 0.23646184 -0.41221912]
[-0.2006352   0.46410143]
[ 0.24873379 -0.44190121]
[-0.21039807  0.48652298]
[ 0.22283884 -0.46369207]
[-0.16725757  0.50943275]
[ 0.19108446 -0.50497843]
[-0.12485053  0.57050893]
[ 0.17570085 -0.50699526]
[-0.15925361  0.55923663]
[ 0.22179892 -0.50868635]
[-0.18736897  0.53524094]
[ 0.21436645 -0.494265  ]
[-0.15832838  0.51988409]
[ 0.22214517 -0.45937384]
[-0.17562893  0.52292825]
[ 0.24478351 -0.47712036]
[-0.22126852  0.50857291]
[ 0.28058193 -0.49278021]
[-0.25189605  0.54869604]
[ 0.25466913 -0.54177505]
[-0.20019122  0.54885661]
[ 0.20658325 -0.50193955]
[-0.19123662  0.50724787]
[ 0.25860862 -0.46065787]
[-0.22272626  0.51350379]
[ 0.22429717 -0.45544946]
[-0.16245056  0.50135987]
[ 0.16361481 -0.47281577]
[-0.16103275  0.51755761]
[ 0.2103838  -0.45699353]
[-0.1881269   0.49941273]
[ 0.19589673 -0.43838085]
[-0.12856655  0.47554439]
[ 0.17766997

...

[-1.60588413 -2.11086889]
[1.61351453 2.14310932]
[-1.58821913 -2.07413919]
[1.64539411 2.14252164]
[-1.61023961 -2.07533255]
[1.63408091 2.09371942]
[-1.59655212 -2.0731304 ]
[1.66227758 2.08531336]
[-1.61587959 -2.08425685]
[1.63283112 2.13187421]
[-1.60799068 -2.07128664]
[1.67003995 2.13346713]
[-1.66986504 -2.0706252 ]
[1.67643867 2.11657937]
[-1.62798323 -2.09333626]
[1.69602448 2.09432869]
[-1.62656806 -2.03840157]
[1.64396185 2.07674791]
[-1.61455922 -2.01469148]
[1.63024253 2.04967983]
[-1.6124629  -2.04809074]
[1.66165162 2.06736847]
[-1.61809514 -2.00996431]
[1.65279941 2.0195536 ]
[-1.63354989 -1.97667585]
[1.64447029 2.02402399]
[-1.60967167 -1.98148623]
[1.67732867 2.02108922]
[-1.62320943 -2.01508286]
[1.66971968 2.05511964]
[-1.60178924 -2.04094133]
[1.62890154 2.08840948]
[-1.61053135 -2.0576886 ]
[1.62836186 2.07245381]
[-1.55993586 -2.06391593]
[1.56141819 2.1082971 ]
[-1.49113335 -2.10143877]
[1.50506905 2.10466811]
[-1.47064546 -2.06235995]
[1.48291131 2.06432751]


[1.31146207 2.1203609 ]
[-1.30759537 -2.11233762]
[1.36508827 2.17319865]
[-1.34422253 -2.15296313]
[1.3932359  2.21811262]
[-1.3245246  -2.16066148]
[1.33737295 2.19379563]
[-1.28444759 -2.14863026]
[1.30980038 2.19613993]
[-1.3031897  -2.16306815]
[1.36231747 2.18337009]
[-1.29369293 -2.13916631]
[1.34535627 2.20902498]
[-1.28371729 -2.14527673]
[1.33241574 2.20479159]
[-1.28505733 -2.15494399]
[1.32664903 2.16863501]
[-1.30932344 -2.12825645]
[1.35401062 2.13387323]
[-1.3390602  -2.12199699]
[1.34719768 2.12738966]
[-1.30862999 -2.08450074]
[1.33585318 2.10426242]
[-1.32284033 -2.07503373]
[1.35392394 2.13963636]
[-1.3081779  -2.07541171]
[1.33798997 2.10348171]
[-1.27697583 -2.08182445]
[1.34136449 2.13718441]
[-1.28880433 -2.06779708]
[1.34512098 2.1076772 ]
[-1.27966497 -2.09986289]
[1.33447513 2.15069187]
[-1.26579865 -2.14563147]
[1.30535305 2.17719143]
[-1.29377571 -2.15417635]
[1.34486586 2.15545175]
[-1.30365502 -2.10571945]
[1.32797299 2.11614815]
[-1.31100201 -2.06600875]


[1.31443282 2.92883315]
[-1.27379958 -2.92741094]
[1.34208182 2.95818796]
[-1.31048012 -2.9509909 ]
[1.37430031 3.00760502]
[-1.33664609 -2.94557005]
[1.39634518 2.96423347]
[-1.38005253 -2.93329194]
[1.38854314 2.97714408]
[-1.36415194 -2.93484516]
[1.41464044 2.94785643]
[-1.40727374 -2.9048539 ]
[1.47655298 2.91390066]
[-1.46835236 -2.86913754]
[1.5065118  2.87460741]
[-1.45028462 -2.81583578]
[1.49942052 2.88503657]
[-1.46157891 -2.8489402 ]
[1.5096812  2.87272658]
[-1.47191458 -2.84040262]
[1.5323021 2.8933565]
[-1.50003944 -2.84170831]
[1.53624916 2.90343981]
[-1.48266799 -2.83862487]
[1.53628741 2.85979881]
[-1.4792737  -2.84784811]
[1.49243946 2.87470564]
[-1.47578047 -2.82794733]
[1.52589351 2.84724911]
[-1.51764996 -2.82736717]
[1.53195769 2.85401403]
[-1.51728896 -2.8338326 ]
[1.57207997 2.84118427]
[-1.5686479  -2.78237013]
[1.62168311 2.78667682]
[-1.58111241 -2.77653717]
[1.60044689 2.81989937]
[-1.55009427 -2.79379643]
[1.5965293  2.79615179]
[-1.5728129  -2.78159947]
[1

[-1.48128374 -2.18137186]
[1.48820149 2.18968105]
[-1.47415949 -2.12088438]
[1.52620413 2.17102385]
[-1.46852458 -2.14746748]
[1.5155949  2.19618449]
[-1.45724727 -2.14384992]
[1.50748827 2.18871173]
[-1.45338128 -2.12999564]
[1.49867192 2.1530377 ]
[-1.48562847 -2.10375883]
[1.54466912 2.11915044]
[-1.53829715 -2.09235522]
[1.55426248 2.09620872]
[-1.50089217 -2.06626565]
[1.5075558  2.11026944]
[-1.47189335 -2.06643148]
[1.53453695 2.09265249]
[-1.53010692 -2.04407752]
[1.53262621 2.06121457]
[-1.51811969 -1.99708692]
[1.57218922 2.03164965]
[-1.52524583 -2.00405771]
[1.5362189  2.04089543]
[-1.48240235 -2.00209943]
[1.50487484 2.0211891 ]
[-1.5018487 -1.9678434]
[1.56160837 1.97568696]
[-1.50981508 -1.93373034]
[1.51914331 1.98932866]
[-1.5000935  -1.98017469]
[1.56673968 2.04470321]
[-1.53900297 -2.02685285]
[1.60359679 2.03271739]
[-1.59937189 -1.99249565]
[1.60044131 2.00757099]
[-1.59569193 -1.94162546]
[1.59876644 1.9952327 ]
[-1.56168443 -1.95189643]
[1.59656434 2.01648246]
[-

[1.7900104 1.7687486]
[-1.75888185 -1.70435712]
[1.82337439 1.71222107]
[-1.75760458 -1.67749603]
[1.79204946 1.7415728 ]
[-1.7796722  -1.70941136]
[1.80866248 1.71312389]
[-1.77114526 -1.69988371]
[1.7966199  1.75492954]
[-1.74838294 -1.71337196]
[1.75347195 1.71457521]
[-1.75182836 -1.64568479]
[1.8027404  1.70290031]
[-1.77327752 -1.65275614]
[1.79625209 1.66504692]
[-1.7941259 -1.6161048]
[1.84496123 1.6285017 ]
[-1.83092971 -1.60134313]
[1.87925756 1.61679428]
[-1.81354176 -1.55291239]
[1.87863434 1.62279153]
[-1.82710468 -1.582958  ]
[1.87535404 1.62232443]
[-1.85303854 -1.60545592]
[1.90281526 1.66483519]
[-1.86972557 -1.61487655]
[1.90221453 1.63338059]
[-1.8375627 -1.6299393]
[1.88354924 1.65776814]
[-1.86897041 -1.61080108]
[1.8884335  1.65120646]
[-1.82455397 -1.59781872]
[1.85023539 1.62917379]
[-1.84183234 -1.61654099]
[1.88333223 1.6790272 ]
[-1.87085394 -1.67039331]
[1.91252212 1.67769386]
[-1.87394658 -1.67569811]
[1.90114915 1.69264829]
[-1.8454982  -1.67649045]
[1.889

In [320]:
print(population_states)

[-0.0168681   0.89042984]


In [261]:
while timestamp <= 2:    
    ionic_noises = noisy_channel(current_noise)
    hidden_states = build_hidden_states(couplings_matrix,population_states,inputs,ionic_noises,I0)
    population_states = update_states(hidden_states,population_states)
    timestamp += dt
    
population_states

array([0.00088421, 0.00164659])

In [260]:
ionic_noises = noisy_channel(current_noise)
print(ionic_noises,'\n')

baseline = couplings_matrix @ population_states
print(baseline + current_noise,'\n')
print(baseline + ionic_noises,'\n')
#print(baseline,'\n',ionic_noises)
print(baseline + inputs + I0 + ionic_noises,'\n')

[0.03681354 0.01177592] 

[0.23232 0.23232] 

[0.26913354 0.24409592] 

[0.56965354 0.54461592] 



array([0.00030601, 0.00053311])

In [100]:
population_states = [S1,S2]
print(population_states)

[1.1, 1.1]


In [17]:
print(couplings)
print(population_states)
print(couplings @ np.transpose(population_states))

[0.2609, 0.0497]
[1.1, 1.1]
0.34166


In [25]:
np.outer(couplings,population_states)

array([[0.28699, 0.28699],
       [0.05467, 0.05467]])

In [148]:
population_states = np.array([10,1])
#print(self_coupling, outer_coupling)
list1 = np.array([self_coupling, -outer_coupling])
#list2 = list1[::-1]

couplings_matrix = [list1,list1[::-1]]
#print()
print(np.shape(couplings_matrix),np.shape(population_states))
#testando = np.multiply(couplings_matrix,population_states)
testando = couplings_matrix @ population_states
#print(couplings_matrix[0],'\n',couplings_matrix[1])
print(testando)
#print(np.sum(testando,0))

(2, 2) (2,)
[ 2.5593 -0.2361]


In [50]:
algo = np.array([1,2])

array([14, 28])

In [238]:
np.shape(np.random.rand(2))

(2,)